In [224]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Задание 1. Производственно-транспортное планирование

## Входные данные:

$productivity_i$ - мощности предприятий (тыс. т) для i = {0, 1, 2, 3, 4}

$unitCost_i$ - удельная стоимость (тыс. р/тыс. т) для i = {0, 1, 2, 3, 4}

$order_i$ - заказы (тыс. т)  i = {0, 1, 2, 3}

$warehouseCapacity_i$ - вместимость склада (тыс. т) для для i = {0, 1, 2}

$D1$ - пропускная способность коммуникаций предприятие-склад

$D3$ - пропускная способность сортировка-потребитель

$C1$ - удельная стоимость перевозки предприятие-склад

$C2$ - удельная стоимость перевозки склад-сортировка

$C3$ - удельная стоимость перевозки сортировка-потребитель


## Переменные

$x_i$ - план производства i-ого завода для i = {0, 1, 2, 3, 4}

$factoryTransfer_{ij} $- объем транспортировки с i-ого завода на j-ый склад

$warehouseTransfer_{ij}$ -объем транспортировки с i-ого склада на j-ую сортировку

$sortingTransfer_{ij}$ - объем транспортировки с i-ой сортировки на j-ого потребителя


## Ограничения

1. $x_i <= productivity_i$ - мощность предприятия не превышена для любого i,j
2. $factoryTransfer_{ij} <= C1_{ij} $- не превышена пропусная способность завод-склад для любого i,j
3. $warehouseTransfer_{ij} <= C2_{ij}$ - не превышена пропусная способность склад-сортировка для любого i,j
4. $sortingTransfer_{ij} <= C3_{ij}$ - не превышена пропусная способность сортировка-потребитель для любого i,j
5. $x_i = factoryTransfer_{i0} + factoryTransfer_{i1} + factoryTransfer_{i2}$ - весь производимый товар транспортируется на склад
6. $\sum_{i=0}^{4} factoryTransfer_{ij} = warehouseTransfer_{j0} + warehouseTransfer_{j1} $ для любого j={0,1,2} - сколько на склад зашло, столько и вышло
7. $\sum_{i=0}^{2} warehouseTransfer_{ij} = sortingTransfer_{j0} + sortingTransfer_{j1} + sortingTransfer_{j2} + sortingTransfer_{j3} $ для любого j={0,1} - сколько на сортировку зашло, столько и вышло
8. $ sortingTransfer_{0j} + sortingTransfer_{1j} = order_j $ для любого j={0,1,2,3}- заказы выполнены 

# 1. Найти план производства и доставки продукции с минимальными суммарными затратами, при котором используются не более двух предприятий.

In [225]:
# считаем цену в тыс. рублей и объем в тыс. тонн
productivity = [600, 500, 700, 400, 600]
unit_cost = [1300, 1500, 1200, 1700, 1100]

order = [100, 200, 200, 500]

warehouse_capacity = [200, 500, 400]

D1 = [[150, 500, 0],
      [100, 0, 400],
      [50, 50, 0],
      [0, 100, 100],
      [100, 0, 50]]

D3 = [[50, 50, 10, 300],
      [70, 150, 200, 250]]

C1 = [[1, 2, 0],
      [2, 0, 1],
      [3, 3, 0],
      [0, 3, 2],
      [2, 0, 3]]
C1 = np.array(C1)*1000

C2 = [[2, 3],
      [1, 4],
      [3, 1]]
C2 = np.array(C2) *1000

C3 = [[2, 1, 3, 4],
      [3, 2, 3, 1]]
C3 = np.array(C3) * 1000

In [226]:
problem = gp.Model('factoryProblem')

x = problem.addVars(5, ub=productivity, name='x')

is_factory_active = problem.addVars(5, vtype=GRB.BINARY, name='is_factory_active')

for i in range(5):
    problem.addConstr(x[i] <= 9999*is_factory_active[i], name="is_factory_active")

problem.addConstr(is_factory_active.sum() <=2)
    
factory_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), (0,2), 
                                                  (1,0), (1,1), (1,2),
                                                  (2,0), (2,1), (2,2),
                                                  (3,0), (3,1), (3,2),
                                                  (4,0), (4,1), (4,2)]), ub=D1, name = 'factory_transfer')

warehouse_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), 
                                                  (1,0), (1,1), 
                                                  (2,0), (2,1)]), name = 'warehouse_transfer')

sorting_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), (0,2), (0,3),
                                                    (1,0), (1,1), (1,2), (1,3) ]), ub=D3, name = 'sorting_transfer')

problem.addConstrs((x[i] == factory_transfer.sum(i, '*') for i in range(5)), name='transfer from factory')

problem.addConstrs((factory_transfer.sum('*', i) <= warehouse_capacity[i]) for i in range(3))


problem.addConstrs((factory_transfer.sum('*', i) == warehouse_transfer.sum(i, '*') for i in range(3)), 
                   name ='transfer from warehouse')

problem.addConstrs((warehouse_transfer.sum('*', i) == sorting_transfer.sum(i, '*') for i in range(2)), 
                   name ='transfer from sorting')

problem.addConstrs((sorting_transfer.sum('*', i) == order[i] for i in range(4)), 
                   name ='orders completed')

obj_fun = gp.LinExpr()
obj_fun += sum(x[i]*unit_cost[i] for i in range(5))
obj_fun += sum(factory_transfer[i, j] * C1[i][j] for i in range(5) for j in range(3))
obj_fun += sum(warehouse_transfer[i, j] * C2[i][j] for i in range(3) for j in range(2))
obj_fun += sum(sorting_transfer[i, j] * C3[i][j] for i in range(2) for j in range(4))

problem.setObjective(obj_fun, GRB.MINIMIZE)
problem.update()
# problem.display()

In [227]:
problem.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 23 rows, 39 columns and 93 nonzeros
Model fingerprint: 0x7b264c2f
Variable types: 34 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+03, 4e+03]
  Bounds range     [1e+00, 7e+02]
  RHS range        [2e+00, 5e+02]
Presolve removed 9 rows and 15 columns
Presolve time: 0.00s
Presolved: 14 rows, 24 columns, 60 nonzeros
Variable types: 19 continuous, 5 integer (5 binary)

Root relaxation: objective 6.805000e+06, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6805000.00    0    2          - 6805000.00      -     -    0s
H    0     0                    6810000.0000 6805000.00  0.07%     -    0s

Explored 1 nodes (11 simplex iterations)

In [228]:
problem.getVars()

[<gurobi.Var x[0] (value 550.0)>,
 <gurobi.Var x[1] (value 450.0)>,
 <gurobi.Var x[2] (value 0.0)>,
 <gurobi.Var x[3] (value 0.0)>,
 <gurobi.Var x[4] (value 0.0)>,
 <gurobi.Var is_factory_active[0] (value 1.0)>,
 <gurobi.Var is_factory_active[1] (value 1.0)>,
 <gurobi.Var is_factory_active[2] (value -0.0)>,
 <gurobi.Var is_factory_active[3] (value -0.0)>,
 <gurobi.Var is_factory_active[4] (value 0.0)>,
 <gurobi.Var factory_transfer[0,0] (value 150.0)>,
 <gurobi.Var factory_transfer[0,1] (value 400.0)>,
 <gurobi.Var factory_transfer[0,2] (value 0.0)>,
 <gurobi.Var factory_transfer[1,0] (value 50.0)>,
 <gurobi.Var factory_transfer[1,1] (value 0.0)>,
 <gurobi.Var factory_transfer[1,2] (value 400.0)>,
 <gurobi.Var factory_transfer[2,0] (value 0.0)>,
 <gurobi.Var factory_transfer[2,1] (value 0.0)>,
 <gurobi.Var factory_transfer[2,2] (value 0.0)>,
 <gurobi.Var factory_transfer[3,0] (value 0.0)>,
 <gurobi.Var factory_transfer[3,1] (value 0.0)>,
 <gurobi.Var factory_transfer[3,2] (value 0.0)>,

# 2. Как изменится план, если на первом складе имеется запас готовой продукции в объеме 100 тыс. тонн?

In [229]:
initial_warehouse_stock = [100, 0, 0]

problem = gp.Model('factoryProblem')

x = problem.addVars(5, ub=productivity, name='x')

factory_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), (0,2), 
                                                  (1,0), (1,1), (1,2),
                                                  (2,0), (2,1), (2,2),
                                                  (3,0), (3,1), (3,2),
                                                  (4,0), (4,1), (4,2)]), ub=D1, name = 'factory_transfer')

warehouse_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), 
                                                  (1,0), (1,1), 
                                                  (2,0), (2,1)]), name = 'warehouse_transfer')

sorting_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), (0,2), (0,3),
                                                    (1,0), (1,1), (1,2), (1,3) ]), ub=D3, name = 'sorting_transfer')

problem.addConstrs((x[i] == factory_transfer.sum(i, '*') for i in range(5)), 
                   name='transfer from factory')

problem.addConstrs((factory_transfer.sum('*', i) <= warehouse_capacity[i]-initial_warehouse_stock[i]) for i in range(3))

problem.addConstrs((factory_transfer.sum('*', i) + initial_warehouse_stock[i] == warehouse_transfer.sum(i, '*') for i in range(3)), 
                   name ='transfer from warehouse')

problem.addConstrs((warehouse_transfer.sum('*', i) == sorting_transfer.sum(i, '*') for i in range(2)), 
                   name ='transfer from sorting')

problem.addConstrs((sorting_transfer.sum('*', i) == order[i] for i in range(4)), 
                   name ='orders completed')

obj_fun = gp.LinExpr()
obj_fun += sum(x[i]*unit_cost[i] for i in range(5))
obj_fun += sum(factory_transfer[i, j] * C1[i][j] for i in range(5) for j in range(3))
obj_fun += sum(warehouse_transfer[i, j] * C2[i][j] for i in range(3) for j in range(2))
obj_fun += sum(sorting_transfer[i, j] * C3[i][j] for i in range(2) for j in range(4))

problem.setObjective(obj_fun, GRB.MINIMIZE)
problem.update()
# problem.display()

In [230]:
problem.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 34 columns and 78 nonzeros
Model fingerprint: 0x57e043d3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 4e+03]
  Bounds range     [1e+01, 7e+02]
  RHS range        [1e+02, 5e+02]
Presolve removed 9 rows and 19 columns
Presolve time: 0.02s
Presolved: 8 rows, 15 columns, 31 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4500000e+06   1.812500e+02   0.000000e+00      0s
       9    6.5200000e+06   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.03 seconds
Optimal objective  6.520000000e+06


In [231]:
problem.getVars()

[<gurobi.Var x[0] (value 500.0)>,
 <gurobi.Var x[1] (value 400.0)>,
 <gurobi.Var x[2] (value 0.0)>,
 <gurobi.Var x[3] (value 0.0)>,
 <gurobi.Var x[4] (value 0.0)>,
 <gurobi.Var factory_transfer[0,0] (value 100.0)>,
 <gurobi.Var factory_transfer[0,1] (value 400.0)>,
 <gurobi.Var factory_transfer[0,2] (value 0.0)>,
 <gurobi.Var factory_transfer[1,0] (value 0.0)>,
 <gurobi.Var factory_transfer[1,1] (value 0.0)>,
 <gurobi.Var factory_transfer[1,2] (value 400.0)>,
 <gurobi.Var factory_transfer[2,0] (value 0.0)>,
 <gurobi.Var factory_transfer[2,1] (value 0.0)>,
 <gurobi.Var factory_transfer[2,2] (value 0.0)>,
 <gurobi.Var factory_transfer[3,0] (value 0.0)>,
 <gurobi.Var factory_transfer[3,1] (value 0.0)>,
 <gurobi.Var factory_transfer[3,2] (value 0.0)>,
 <gurobi.Var factory_transfer[4,0] (value 0.0)>,
 <gurobi.Var factory_transfer[4,1] (value 0.0)>,
 <gurobi.Var factory_transfer[4,2] (value 0.0)>,
 <gurobi.Var warehouse_transfer[0,0] (value 0.0)>,
 <gurobi.Var warehouse_transfer[0,1] (value 

## 3. Найти оптимальный план, если затраты на организацию производства составляют 1, 2, 3, 2.5 и 1.5 млн. руб.

Введем бинарные переменные is_factory_active

In [232]:
activation_cost = [10**3, 2*10**3, 3*10**3, 2.5*10**3, 1.5*10**3] #тыс.р

problem = gp.Model('factoryProblem')

x = problem.addVars(5, ub=productivity, name='x')

is_factory_active = problem.addVars(5, vtype=GRB.BINARY, name='is_factory_active')

for i in range(5):
    problem.addConstr(x[i] <= 9999*is_factory_active[i], name="is_factory_active")

factory_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), (0,2), 
                                                  (1,0), (1,1), (1,2),
                                                  (2,0), (2,1), (2,2),
                                                  (3,0), (3,1), (3,2),
                                                  (4,0), (4,1), (4,2)]), ub=D1, name = 'factory_transfer')

warehouse_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), 
                                                  (1,0), (1,1), 
                                                  (2,0), (2,1)]), name = 'warehouse_transfer')

sorting_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), (0,2), (0,3),
                                                    (1,0), (1,1), (1,2), (1,3) ]), ub=D3, name = 'sorting_transfer')

problem.addConstrs((x[i] == factory_transfer.sum(i, '*') for i in range(5)), 
                   name='transfer from factory')

problem.addConstrs((factory_transfer.sum('*', i) <= warehouse_capacity[i]) for i in range(3))

problem.addConstrs((factory_transfer.sum('*', i) == warehouse_transfer.sum(i, '*') for i in range(3)), 
                   name ='transfer from warehouse')

problem.addConstrs((warehouse_transfer.sum('*', i) == sorting_transfer.sum(i, '*') for i in range(2)), 
                   name ='transfer from sorting')

problem.addConstrs((sorting_transfer.sum('*', i) == order[i] for i in range(4)), 
                   name ='orders completed')

obj_fun = gp.LinExpr()
obj_fun += sum(x[i]*unit_cost[i] for i in range(5))
obj_fun += sum(factory_transfer[i, j] * C1[i][j] for i in range(5) for j in range(3))
obj_fun += sum(warehouse_transfer[i, j] * C2[i][j] for i in range(3) for j in range(2))
obj_fun += sum(sorting_transfer[i, j] * C3[i][j] for i in range(2) for j in range(4))
obj_fun += sum(activation_cost[i] * is_factory_active[i] for i in range(5))

problem.setObjective(obj_fun, GRB.MINIMIZE)
problem.update()
# problem.display()

In [233]:
problem.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22 rows, 39 columns and 88 nonzeros
Model fingerprint: 0x5c79f287
Variable types: 34 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+03, 4e+03]
  Bounds range     [1e+00, 7e+02]
  RHS range        [1e+02, 5e+02]
Presolve removed 9 rows and 15 columns
Presolve time: 0.00s
Presolved: 13 rows, 24 columns, 55 nonzeros
Variable types: 19 continuous, 5 integer (5 binary)

Root relaxation: objective 6.793417e+06, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6793416.67    0    2          - 6793416.67      -     -    0s
H    0     0                    6794500.0000 6793416.67  0.02%     -    0s

Cutting planes:
  Relax-and-lift: 3

Exp

In [234]:
problem.getVars()

[<gurobi.Var x[0] (value 550.0)>,
 <gurobi.Var x[1] (value 400.0)>,
 <gurobi.Var x[2] (value 0.0)>,
 <gurobi.Var x[3] (value 0.0)>,
 <gurobi.Var x[4] (value 50.0)>,
 <gurobi.Var is_factory_active[0] (value 1.0)>,
 <gurobi.Var is_factory_active[1] (value 1.0)>,
 <gurobi.Var is_factory_active[2] (value -0.0)>,
 <gurobi.Var is_factory_active[3] (value -0.0)>,
 <gurobi.Var is_factory_active[4] (value 1.0)>,
 <gurobi.Var factory_transfer[0,0] (value 150.0)>,
 <gurobi.Var factory_transfer[0,1] (value 400.0)>,
 <gurobi.Var factory_transfer[0,2] (value 0.0)>,
 <gurobi.Var factory_transfer[1,0] (value 0.0)>,
 <gurobi.Var factory_transfer[1,1] (value 0.0)>,
 <gurobi.Var factory_transfer[1,2] (value 400.0)>,
 <gurobi.Var factory_transfer[2,0] (value 0.0)>,
 <gurobi.Var factory_transfer[2,1] (value 0.0)>,
 <gurobi.Var factory_transfer[2,2] (value 0.0)>,
 <gurobi.Var factory_transfer[3,0] (value 0.0)>,
 <gurobi.Var factory_transfer[3,1] (value 0.0)>,
 <gurobi.Var factory_transfer[3,2] (value 0.0)>,

## 4. Найти оптимальный план, если удельная стоимость хранения на складах составляет 2, 1.5 и 3 тыс. руб за тонну.

In [235]:
warehouse_price = [2*10**3, 1.5*10**3, 3*10**3] #тыс. руб/тыс.т

problem = gp.Model('factoryProblem')

x = problem.addVars(5, ub=productivity, name='x')

factory_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), (0,2), 
                                                  (1,0), (1,1), (1,2),
                                                  (2,0), (2,1), (2,2),
                                                  (3,0), (3,1), (3,2),
                                                  (4,0), (4,1), (4,2)]), ub=D1, name = 'factory_transfer')

warehouse_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), 
                                                  (1,0), (1,1), 
                                                  (2,0), (2,1)]), name = 'warehouse_transfer')

sorting_transfer = problem.addVars (gp.tuplelist([(0,0), (0,1), (0,2), (0,3),
                                                    (1,0), (1,1), (1,2), (1,3) ]), ub=D3, name = 'sorting_transfer')

problem.addConstrs((x[i] == factory_transfer.sum(i, '*') for i in range(5)), 
                   name='transfer from factory')

problem.addConstrs((factory_transfer.sum('*', i) <= warehouse_capacity[i]) for i in range(3))

problem.addConstrs((factory_transfer.sum('*', i) == warehouse_transfer.sum(i, '*') for i in range(3)), 
                   name ='transfer from warehouse')

problem.addConstrs((warehouse_transfer.sum('*', i) == sorting_transfer.sum(i, '*') for i in range(2)), 
                   name ='transfer from sorting')

problem.addConstrs((sorting_transfer.sum('*', i) == order[i] for i in range(4)), 
                   name ='orders completed')

obj_fun = gp.LinExpr()
obj_fun += sum(x[i]*unit_cost[i] for i in range(5))
obj_fun += sum(factory_transfer[i, j] * C1[i][j] for i in range(5) for j in range(3))
obj_fun += sum(warehouse_transfer[i, j] * C2[i][j] for i in range(3) for j in range(2))
obj_fun += sum(sorting_transfer[i, j] * C3[i][j] for i in range(2) for j in range(4))
obj_fun += sum(factory_transfer[i, j] * warehouse_price[j] for i in range(5) for j in range(3))

problem.setObjective(obj_fun, GRB.MINIMIZE)
problem.update()
# problem.display()

In [236]:
problem.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 34 columns and 78 nonzeros
Model fingerprint: 0x31aee795
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 6e+03]
  Bounds range     [1e+01, 7e+02]
  RHS range        [1e+02, 5e+02]
Presolve removed 9 rows and 17 columns
Presolve time: 0.01s
Presolved: 8 rows, 17 columns, 35 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4500000e+06   1.687500e+02   0.000000e+00      0s
       8    8.9900000e+06   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds
Optimal objective  8.990000000e+06


In [237]:
problem.getVars()

[<gurobi.Var x[0] (value 550.0)>,
 <gurobi.Var x[1] (value 400.0)>,
 <gurobi.Var x[2] (value 0.0)>,
 <gurobi.Var x[3] (value 0.0)>,
 <gurobi.Var x[4] (value 50.0)>,
 <gurobi.Var factory_transfer[0,0] (value 150.0)>,
 <gurobi.Var factory_transfer[0,1] (value 400.0)>,
 <gurobi.Var factory_transfer[0,2] (value 0.0)>,
 <gurobi.Var factory_transfer[1,0] (value 0.0)>,
 <gurobi.Var factory_transfer[1,1] (value 0.0)>,
 <gurobi.Var factory_transfer[1,2] (value 400.0)>,
 <gurobi.Var factory_transfer[2,0] (value 0.0)>,
 <gurobi.Var factory_transfer[2,1] (value 0.0)>,
 <gurobi.Var factory_transfer[2,2] (value 0.0)>,
 <gurobi.Var factory_transfer[3,0] (value 0.0)>,
 <gurobi.Var factory_transfer[3,1] (value 0.0)>,
 <gurobi.Var factory_transfer[3,2] (value 0.0)>,
 <gurobi.Var factory_transfer[4,0] (value 50.0)>,
 <gurobi.Var factory_transfer[4,1] (value 0.0)>,
 <gurobi.Var factory_transfer[4,2] (value 0.0)>,
 <gurobi.Var warehouse_transfer[0,0] (value 0.0)>,
 <gurobi.Var warehouse_transfer[0,1] (valu